In [2]:
from common_functions import ret_metabase, google_sheets, upload_dataframe_to_snowflake, snowflake_query, upload_dataframe_to_pg, dwh_query, get_secret
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
# import psycopg2
import numpy as np
import gspread
import sqlalchemy
import pytz
import psycopg2

In [3]:
campaign_offers = dwh_query("egypt", "select * from materialized_views.offer_tool_logs", "read")
campaign_offers

/home/ec2-user/service_account_key.json


,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
0,1,11,NaN,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Pending,2025-09-15 16:12:05,2025-09-16 02:12:05
1,2,11,153.0,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Approved,2025-09-15 17:05:20,2025-09-16 03:05:20
2,3,12,153.0,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Approved,2025-09-15 17:20:07,2025-09-16 03:20:07
3,4,13,36.0,"[42031, 187880, 152179, 220400, 21221, 600256,...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
4,5,13,36.0,"[42031, 187880, 152179, 220400, 21221, 600256,...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
5,6,14,20.0,"[189262, 600256, 42031, 40069, 652631, 21221, ...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
6,7,15,20.0,"[42031, 89747, 189262, 180832, 21221, 304614, ...",SKU_discounts,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
7,8,16,376.0,"[42031, 258853, 456325, 21221, 205682, 787231,...",SKU_discounts,Pending,2025-09-17 14:10:28,2025-09-18 00:10:28
8,9,17,739.0,"[42031, 69636, 45131, 215258, 463672, 673304, ...",SKU_discounts,Pending,2025-09-18 12:00:00,2025-09-18 22:00:00
9,11,19,110.0,"[42031, 457975, 313057, 21221, 156922, 36959, ...",SKU_discounts,Pending,2025-09-18 13:43:19,2025-09-18 23:43:19


In [5]:
import pandas as pd
campaign_offers = campaign_offers[campaign_offers['offer_type'] == 'offers']
# Make sure end_at is a datetime column
campaign_offers['end_at'] = pd.to_datetime(campaign_offers['end_at'])

# Get current datetime
now = pd.Timestamp.now()

# Filter rows where end_at is in the future
campaign_ongoing = campaign_offers[campaign_offers['end_at'] > now]
campaign_ongoing


,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
25,26,34,36.0,"[669974, 114999, 20102, 179817, 319941, 438891...",offers,Pending,2025-10-21 12:50:27,2025-10-24 12:50:27


In [7]:
campaign_ids = campaign_ongoing['campaign_id'].tolist()
print(campaign_ids)


[34]


In [8]:
offers_id = dwh_query("egypt", "select * from materialized_views.offer_tool_logs", "read")
offers_id

/home/ec2-user/service_account_key.json


,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
0,1,11,NaN,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Pending,2025-09-15 16:12:05,2025-09-16 02:12:05
1,2,11,153.0,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Approved,2025-09-15 17:05:20,2025-09-16 03:05:20
2,3,12,153.0,"[42031, 770467, 390843, 471035, 21221, 373805,...",SKU_discounts,Approved,2025-09-15 17:20:07,2025-09-16 03:20:07
3,4,13,36.0,"[42031, 187880, 152179, 220400, 21221, 600256,...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
4,5,13,36.0,"[42031, 187880, 152179, 220400, 21221, 600256,...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
5,6,14,20.0,"[189262, 600256, 42031, 40069, 652631, 21221, ...",offers,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
6,7,15,20.0,"[42031, 89747, 189262, 180832, 21221, 304614, ...",SKU_discounts,Pending,2025-09-17 12:00:00,2025-09-17 22:00:00
7,8,16,376.0,"[42031, 258853, 456325, 21221, 205682, 787231,...",SKU_discounts,Pending,2025-09-17 14:10:28,2025-09-18 00:10:28
8,9,17,739.0,"[42031, 69636, 45131, 215258, 463672, 673304, ...",SKU_discounts,Pending,2025-09-18 12:00:00,2025-09-18 22:00:00
9,11,19,110.0,"[42031, 457975, 313057, 21221, 156922, 36959, ...",SKU_discounts,Pending,2025-09-18 13:43:19,2025-09-18 23:43:19


In [13]:
offers_filtered = offers_id[offers_id['campaign_id'].isin(campaign_ids)]
offers_filtered

,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
25,26,34,36.0,"[669974, 114999, 20102, 179817, 319941, 438891...",offers,Pending,2025-10-21 12:50:27,2025-10-24 12:50:27


In [21]:
import pandas as pd

# Example: updating the 'end_at' value for a specific row
offers_filtered.loc[offers_filtered['id'] == 26, 'end_at'] = pd.Timestamp('2025-10-21 7:17:27')
offers_filtered

,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
25,26,34,36.0,"[669974, 114999, 20102, 179817, 319941, 438891...",offers,Pending,2025-10-21 12:50:27,2025-10-21 07:17:27


In [22]:
offers_filtered['end_at'] = pd.to_datetime(offers_filtered['end_at'])

# Get current datetime
now = pd.Timestamp.now()

# Filter rows where end_at is in the future
offers_ended = offers_filtered[offers_filtered['end_at'] < now]
offers_ended

/tmp/ipykernel_5519/215201034.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offers_filtered['end_at'] = pd.to_datetime(offers_filtered['end_at'])


,id,campaign_id,supplier_id,retailer_list,offer_type,status,start_at,end_at
25,26,34,36.0,"[669974, 114999, 20102, 179817, 319941, 438891...",offers,Pending,2025-10-21 12:50:27,2025-10-21 07:17:27


In [25]:
# Extract the values from offers_ended DataFrame first
if not offers_ended.empty:
    campaign_id_value = offers_ended['campaign_id'].iloc[0]
    supplier_id_value = offers_ended['supplier_id'].iloc[0]
    retailer_id_value = offers_ended['retailer_list'].iloc[0]
    print(f"campaign ID: {campaign_id_value}")
    print(f"Supplier ID: {supplier_id_value}")
    print(f"Retailer ID: {retailer_id_value}")
else:
    print("offers_ended is empty!")


campaign ID: 34
Supplier ID: 36.0
Retailer ID: [669974, 114999, 20102, 179817, 319941, 438891, 152504, 20378, 98617, 184557, 328030, 401755, 608384, 591725, 234212, 190141, 897730, 164698, 158267, 196132, 296222, 635153, 151822, 234304, 521814, 9826, 489128, 214581, 113023, 360562, 380649, 869843, 288673, 132183, 732092, 943802, 504998, 871927, 152598, 238268, 527392, 341763, 134122, 364174, 412875, 391234, 230104, 342497, 411050, 506812, 928694, 152802, 23553, 547912, 9163, 124851, 534321, 220809, 269134, 599447, 332365, 355001, 748009, 326155, 35850, 944592, 301342, 490644, 748947, 199911, 80719, 115532, 38110, 453555, 768536, 294757, 111929, 666819, 155827, 939906, 499538, 25330, 235893, 720849, 680107, 567032, 213854, 378211, 536939, 26641, 208087]


In [41]:
query1 = """with offer_redeemers as (
select 	retailer_id,
    	maxab_offer_id,
    	count (distinct parent_sales_order_id) as po_count,
    	sum (discount) as redeemed_value, 
		max(discount) as discount_value
from public.sales_orders so 
where sales_order_status_id in (6,9) 
and maxab_offer_id is not null 
and discount > 0 
and date_trunc(month, so.created_at::date) >= date_trunc ('month',current_date) - interval '6 month'
group by all),

sup_products as (select  distinct sps.parent_supplier_id as supplier_id, 
		ps.product_id
from supplier_parent_suppliers sps
left join product_supplier ps on ps.supplier_id = sps.supplier_id 
left join products p on p.id = ps.product_id 
left join categories c on c.id = p.category_id ),

offer as(
select  s.*, 
		po_count as redeemed_orders, 
		redeemed_value, 
		discount_value
from (
select s.*, 
		TICKET_SIZE_MINIMUM_VALUE
from (
select 	
		SPLIT_PART(description, '_', 2)      AS campaign_id,  
    	TRY_CAST(REGEXP_SUBSTR(description, '\\d+', 1, 2) AS INT) AS supplier_id, 
		offer.id as offer_id, 
		offer.active_from as start_at,
		offer.active_to as end_at, 
    	retailer.value::int as retailer_id, 
		rule_id, 
		description,
		dense_rank() over(partition by campaign_id order by start_at asc) offer_number,
		case when description like '%ch_r%' then 'Churned Recent'
		when description like '%act_c%' then 'Active Competition' 
		when description like '%act%' then 'Active'
		when description like '%ch_o%' then 'Churned Old' end as retailer_tag
from offers.offer, 
    lateral flatten(strtok_to_array(replace(replace(WHITE_LIST_RETAILER_IDS,'{', ''),'}',''),',')) retailer
where start_at::date between current_date - 90 and current_date
and TRY_CAST(REGEXP_SUBSTR(description, '\\d+', 1, 2) AS INT) IS NOT NULL"""
query2 = f"""
and campaign_id = 34
and is_cancelled = false) s 
left join offers.rule r on r.parent_rule_id = s.rule_id and r.dtype = 'TicketSizeRule' )s
left join offer_redeemers ofr on ofr.retailer_id = s.retailer_id and s.offer_id = ofr.maxab_offer_id 
order by s.retailer_id, start_At)

select campaign_id, 
		supplier_id,
        start_at, 
		end_at,
		-- count(distinct product_id) as products_in_campaign, 
		-- retailer_tag, 
		count(distinct retailer_id) as number_retailers, 
	    count(distinct start_at) as number_offers, 
	    count(distinct case when product_nmv_post > 0 then retailer_id end) as converted_retailers, 
	    count(distinct case when product_nmv_lm > 0 then retailer_id end) as converted_prior, 
	   	sum(product_nmv_post) as product_nmv, 
	   	sum(product_nmv_lm) as product_nmv_prior, 
	   	sum(redeemed_value) as total_burn, 
	   	converted_retailers/number_retailers as conversion_rate, 
	   	converted_prior/number_retailers as conversion_rate_pre
from (
select  campaign_id,
		supplier_id, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) start_at, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo', end_at) end_at,
		offer_number, 
		s.retailer_id, 
		retailer_tag, 
		redeemed_orders, 
		redeemed_value, 
		discount_value, 
		min_gmv, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) - interval '1 month' as start_lm, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo', end_at) - interval '1 month' as end_lm, 
		sum(case when p.id is not null and so.created_at between start_at and end_at then coalesce(pso.total_price, 0) else 0 end) as product_nmv_post, 
		sum(case when p.id is not null and so.created_at between start_lm and end_lm then coalesce(pso.total_price, 0) else 0 end) as product_nmv_lm, 
		s.type_id as brand_id 
from (
select  offer.*, 
		type_id, 
		min_gmv, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) - interval '1 month' as start_lm, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo', end_at) - interval '1 month' as end_lm 
from offer 
left join (
select distinct rule.parent_rule_id, coalesce(allb.type_id,oneb.type_id) as type_id,ticket_size_minimum_value as off_ts, oneb.minimum_gmv as min_gmv
from offers.rule
left join offers.one_of_brands_rule_item as oneb on oneb.one_of_brands_rule_id = rule.id and dtype = 'OneOfBrandsRule'
left join offers.all_of_brands_rule_item as allb on allb.all_of_brands_rule_id = rule.id and dtype = 'AllOfBrandsRule') s on s.parent_rule_id = offer.rule_id 
where type_id is not null 
order by retailer_id ) s 
left join sales_orders so on so.retailer_id = s.retailer_id and (CONVERT_TIMEZONE('UTC', 'Africa/Cairo',so.created_at) between CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) and CONVERT_TIMEZONE('UTC', 'Africa/Cairo',end_at) or CONVERT_TIMEZONE('UTC', 'Africa/Cairo',so.created_at)  between start_lm and end_lm)
left join product_sales_order pso on pso.sales_order_id = so.id and purchased_item_count > 0 
left join products p on p.id = pso.product_id and p.brand_id = s.type_id 
group by all 
order by discount_value ) 

group by all """
performance = snowflake_query("egypt", f"""
{query1}{query2}
""")
performance

/home/ec2-user/service_account_key.json


,campaign_id,supplier_id,start_at,end_at,number_retailers,number_offers,converted_retailers,converted_prior,product_nmv,product_nmv_prior,total_burn,conversion_rate,conversion_rate_pre


In [42]:
performance = snowflake_query("egypt", """
with offer_redeemers as (
select 	retailer_id,
    	maxab_offer_id,
    	count (distinct parent_sales_order_id) as po_count,
    	sum (discount) as redeemed_value, 
		max(discount) as discount_value
from public.sales_orders so 
where sales_order_status_id in (6,9) 
and maxab_offer_id is not null 
and discount > 0 
and date_trunc(month, so.created_at::date) >= date_trunc ('month',current_date) - interval '6 month'
group by all),

sup_products as (select  distinct sps.parent_supplier_id as supplier_id, 
		ps.product_id
from supplier_parent_suppliers sps
left join product_supplier ps on ps.supplier_id = sps.supplier_id 
left join products p on p.id = ps.product_id 
left join categories c on c.id = p.category_id ),

offer as(
select  s.*, 
		po_count as redeemed_orders, 
		redeemed_value, 
		discount_value
from (
select s.*, 
		TICKET_SIZE_MINIMUM_VALUE
from (
select 	
		SPLIT_PART(description, '_', 2)      AS campaign_id,  
    	REGEXP_SUBSTR(description, '\\d+', 1, 2)  AS supplier_id, 
		offer.id as offer_id, 
		offer.active_from as start_at,
		offer.active_to as end_at, 
    	retailer.value::int as retailer_id, 
		rule_id, 
		description,
		dense_rank() over(partition by campaign_id order by start_at asc) offer_number,
		case when description like '%ch_r%' then 'Churned Recent'
		when description like '%act_c%' then 'Active Competition' 
		when description like '%act%' then 'Active'
		when description like '%ch_o%' then 'Churned Old' end as retailer_tag
from offers.offer, 
    lateral flatten(strtok_to_array(replace(replace(WHITE_LIST_RETAILER_IDS,'{', ''),'}',''),',')) retailer
where start_at::date between current_date - 90 and current_date
and campaign_id = 34
and is_cancelled = false) s 
left join offers.rule r on r.parent_rule_id = s.rule_id and r.dtype = 'TicketSizeRule' )s
left join offer_redeemers ofr on ofr.retailer_id = s.retailer_id and s.offer_id = ofr.maxab_offer_id 
order by s.retailer_id, start_At)

select campaign_id, 
		supplier_id,
        start_at, 
		end_at,
		-- count(distinct product_id) as products_in_campaign, 
		-- retailer_tag, 
		count(distinct retailer_id) as number_retailers, 
	    count(distinct start_at) as number_offers, 
	    count(distinct case when product_nmv_post > 0 then retailer_id end) as converted_retailers, 
	    count(distinct case when product_nmv_lm > 0 then retailer_id end) as converted_prior, 
	   	sum(product_nmv_post) as product_nmv, 
	   	sum(product_nmv_lm) as product_nmv_prior, 
	   	sum(redeemed_value) as total_burn, 
	   	converted_retailers/number_retailers as conversion_rate, 
	   	converted_prior/number_retailers as conversion_rate_pre
from (
select  campaign_id,
		supplier_id, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) start_at, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo', end_at) end_at,
		offer_number, 
		s.retailer_id, 
		retailer_tag, 
		redeemed_orders, 
		redeemed_value, 
		discount_value, 
		min_gmv, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) - interval '1 month' as start_lm, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo', end_at) - interval '1 month' as end_lm, 
		sum(case when p.id is not null and so.created_at between start_at and end_at then coalesce(pso.total_price, 0) else 0 end) as product_nmv_post, 
		sum(case when p.id is not null and so.created_at between start_lm and end_lm then coalesce(pso.total_price, 0) else 0 end) as product_nmv_lm, 
		s.type_id as brand_id 
from (
select  offer.*, 
		type_id, 
		min_gmv, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) - interval '1 month' as start_lm, 
		CONVERT_TIMEZONE('UTC', 'Africa/Cairo', end_at) - interval '1 month' as end_lm 
from offer 
left join (
select distinct rule.parent_rule_id, coalesce(allb.type_id,oneb.type_id) as type_id,ticket_size_minimum_value as off_ts, oneb.minimum_gmv as min_gmv
from offers.rule
left join offers.one_of_brands_rule_item as oneb on oneb.one_of_brands_rule_id = rule.id and dtype = 'OneOfBrandsRule'
left join offers.all_of_brands_rule_item as allb on allb.all_of_brands_rule_id = rule.id and dtype = 'AllOfBrandsRule') s on s.parent_rule_id = offer.rule_id 
where type_id is not null 
order by retailer_id ) s 
left join sales_orders so on so.retailer_id = s.retailer_id and (CONVERT_TIMEZONE('UTC', 'Africa/Cairo',so.created_at) between CONVERT_TIMEZONE('UTC', 'Africa/Cairo',start_at) and CONVERT_TIMEZONE('UTC', 'Africa/Cairo',end_at) or CONVERT_TIMEZONE('UTC', 'Africa/Cairo',so.created_at)  between start_lm and end_lm)
left join product_sales_order pso on pso.sales_order_id = so.id and purchased_item_count > 0 
left join products p on p.id = pso.product_id and p.brand_id = s.type_id 
group by all 
order by discount_value ) 

group by all 
""")
performance

/home/ec2-user/service_account_key.json


ERROR:common_functions:An error occurred: 100038 (22018): Numeric value '' is not recognized
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/git_files/Seif/Seif's_stuff/common_functions.py", line 429, in snowflake_query
    cur.execute(query)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/snowflake/connector/cursor.py", line 1121, in execute
    Error.errorhandler_wrapper(self.connection, self, error_class, errvalue)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/snowflake/connector/errors.py", line 286, in errorhandler_wrapper
    handed_over = Error.hand_to_other_handler(
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/snowflake/connector/errors.py", line 341, in hand_to_other_handler
    cursor.errorhandler(connection, cursor, error_class, error_value)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/snowflake/connector/errors.py", line 217, in default_err

ProgrammingError: 100038 (22018): Numeric value '' is not recognized